In [8]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append(os.path.abspath("") + "/../src/pynirs")

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate

from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

from nirs_set import NIRS_Set
import preprocessor as pp
import splitter as sp
import augmenter as aug
from pipeline_tools import FeatureUnionOnAxis


#### MODEL creation
def create_model(optimizer = 'adagrad', 
                 kernel_initializer = 'glorot_uniform', 
                 dropout = 0.2):
    model = Sequential()
    model.add(Dense(64, activation = 'relu', kernel_initializer = kernel_initializer))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation = 'sigmoid', kernel_initializer = kernel_initializer))

    model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    return model


#### Data augmenters

# # augmentation definition (default uses all functions)
augmenter = aug.Augmenter([
    ('spl_def', aug.RandomSplineDeformation()), 
    ('rot_trans', aug.RotateTranslate()), 
    ('spl_spl', aug.SplineSimplification())
    ], keep_classes = True)


#### Pipeline transformers

# # Order 1 filters stacked
filters = FeatureUnionOnAxis([
    ('id', pp.IdentityTransformer()), 
    ('derivate', pp.Derivate()), 
    ('spl', pp.RobustNormalVariate()), 
    ('haar', pp.Wavelet())
    ], axis = 1)

# # Order 2 filters stacked
sndfilters = FeatureUnionOnAxis([
    ('id', pp.IdentityTransformer()), 
    ('derivate', pp.Derivate()), 
    ('spl', pp.RobustNormalVariate()), 
    ('haar', pp.Wavelet())
    ], axis = 1)

# # The pipeline with the keras regressor. Can be replaced by any sklearn regressor if
# # filters preprocessing is done in 1D (FeatureUnion instead of FeatureUnionOnAxis)
pipeline = Pipeline([
    ('scaler', MinMaxScaler()), 
    ('filters', filters), 
    ('sndfilters', sndfilters),
    # ('pls', PLS())
    ('nn', KerasRegressor(build_fn = create_model, verbose = 0))
])

# ## TransformedTargetRegressor enables the transformation of Y before and after
estimator = TransformedTargetRegressor(estimator = pipeline, transformer = StandardScaler())


#########

n = NIRS_Set('data')
X, y = n.load('Xcal.csv', 'Ycal.csv', x_hdr = 0, y_hdr = 0)
print(X.shape, y.shape)


# ### EXAMPLE 1 > simple fit
X_train, X_test, y_train, y_test = sp.sk_train_test_split(X, y)
X_train, y_train = augmenter.fit_transform(X_train, y_train)
estimator.fit(X_train, y_train)
Y_preds = estimator.predict(X_test)
print(f1_score(y_test, Y_preds, average = 'micro'))



# ### EXAMPLE 2 > Cross Validation no augmentation
kf = sp.KFold(method='kennard_stone', n_splits=5)
print(cross_validate(estimator, X, y, scoring = 'neg_mean_squared_error', cv = kf))

# ### EXAMPLE 3 > Hyperparams
param_grid = {
    'haar__method':['haar', 'bior1.3'], 
    'nn__optimizer':['rmsprop', 'adam', 'adagrad'], 
    'nn__epochs':[4, 8], 
    'nn__dropout':[0.1, 0.2], 
    'nn__kernel_initializer':['glorot_uniform', 'normal', 'uniform']
}

grid = GridSearchCV(pipeline, cv = 3, param_grid = param_grid)
grid.fit(X_train, y_train)
print("Best: %f using %s" % (grid.best_score_, grid.best_params_))
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


AttributeError: module 'augmenter' has no attribute 'Augmenter'